Create a Jupyter Notebook file called climate_analysis.ipynb and use it to complete your climate analysis and data exporation.

Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.

Use SQLAlchemy create_engine to connect to your sqlite database.

Use SQLAlchemy automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.

In [9]:
# Dependencies
# ----------------------------------
import pandas as pd

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()


In [2]:
# Create Database Connection
# ----------------------------------

engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

In [5]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [6]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurements', 'stations']

In [10]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [11]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [12]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['measurements', 'stations']

In [13]:
# Assign the demographics class to a variable called `Demographics`
Measurement = Base.classes.measurements
Station = Base.classes.stations

In [14]:
# Create a session
session = Session(engine)

In [15]:
# Use the session to query Demographics table and display the first 5 locations
for row in session.query(Station, Station.name).limit(8).all():
    print(row)

(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA160>, 'WAIKIKI 717.2, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA1D0>, 'KANEOHE 838.1, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA240>, 'KUALOA RANCH HEADQUARTERS 886.9, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA2B0>, 'PEARL CITY, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA320>, 'UPPER WAHIAWA 874.3, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA390>, 'WAIMANALO EXPERIMENTAL FARM, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA438>, 'WAIHEE 837.5, HI US')
(<sqlalchemy.ext.automap.stations object at 0x000002D3BEBCA4E0>, 'HONOLULU OBSERVATORY 702.2, HI US')
